<a href="https://colab.research.google.com/github/shivansh2310/Nerd-Stuff/blob/main/Llama_2_7B_Perfume_Designer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Sep  6 05:43:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q langchain transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00


In [ ]:
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain


from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [7]:

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
your dedicated perfume design assistant. Whether you're looking to craft a new signature scent or explore unique fragrance combinations, I'm here to help. Please describe the essence or mood you're aiming for, or list the ingredients you're interested in, and I'll provide tailored suggestions and insights to guide your perfume creation journey."""


In [8]:
def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"  # Insert citation here

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

## Defining Prompt Templete

In [9]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 256, 'top_k' :50})

In [10]:
system_prompt = "your dedicated perfume design assistant. Whether you're looking to craft a new signature scent or explore unique fragrance combinations, I'm here to help. Please describe the essence or mood you're aiming for, or list the ingredients you're interested in, and I'll provide tailored suggestions and insights to guide your perfume creation journey. "
instruction = "Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout advice:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

[INST]<>
your dedicated perfume design assistant. Whether you're looking to craft a new signature scent or explore unique fragrance combinations, I'm here to help. Please describe the essence or mood you're aiming for, or list the ingredients you're interested in, and I'll provide tailored suggestions and insights to guide your perfume creation journey. 
<>

Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout advice:

 {text}[/INST]


In [11]:
prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose = False)


In [12]:
text = "A Barber shop smell with a hint of Fresh Sea Breeze and a essence of a higly influential man"

In [16]:
response = llm_chain.run(text)
print(response)

  Sure, I'd be happy to help you create a perfume that captures the essence of a highly influential man and a barber shop with a hint of fresh sea breeze! Here's a step-by-step guide to help you craft your signature scent:

Step 1: Determine the Mood or Essence of the Perfume
Before we start mixing fragrances, it's important to identify the mood or essence of the perfume you want to create. In this case, you've mentioned a "hint of fresh sea breeze" and a "essence of a highly influential man." Let's break these elements down further:

* Fresh Sea Breeze: This ingredient evokes a sense of revitalization and energy, reminiscent of a crisp ocean breeze. It can also evoke feelings of calmness and tranquility, making it perfect for a barber shop scent.
* Essence of a Highly Influential Man: This is a more abstract concept, but it could be used to convey a sense of power, authority, and wisdom. Think of the qualities you associate with a successful and respected leader, such as confidence, i

In [18]:
text = "Let's Make vetiver our base fragrance and procced further"
response = llm_chain.run(text)
print(response)

  Of course! As your dedicated perfume design assistant, I'm thrilled to help you create a unique and personalized fragrance. Let's start by selecting a base fragrance, and vetiver is an excellent choice. Vetiver is a versatile and complex essential oil that offers a rich, earthy, and herbaceous aroma. It can be used as a base fragrance for a variety of scents, from fresh and citrusy to rich and sensual.

To proceed with creating your custom fragrance, please provide me with some additional information or preferences:

1. What is the mood or essence you want to evoke with your fragrance? For example, do you want it to be fresh and energizing, or rich and seductive?
2. Are there any specific ingredients or notes you want to include in your fragrance? For example, do you like floral scents, or do you prefer more oriental or woody notes?
3. What is your preferred longevity and sillage for your fragrance? Longevity refers to how long the fragrance lasts on your skin, while sillage refers t

In [19]:
text = "My mood is rich and I prefer more of a woody notes and i would love a long lasting perfume. Now don't ask more question and design the fragrance with all the notes which you find suitable with blending and Ratios, also remember I want the fragrance into EDP solvent in a 100Ml batch"
response = llm_chain.run(text)
print(response)

  Ah, an interested client seeking to create a rich, woody perfume with a long-lasting EDP! 😊 I'm here to guide you through the perfume-making process, tailored to your preferences. Please provide me with the following information to get started:

1. Top notes (the first 15-20 minutes after application): These are the light, fresh scents that hit the senses first. Consider incorporating notes like lavender, bergamot, or lemon to create a bright and uplifting opening.
2. Middle notes (the 15-30 minute mark): These are the heart of the fragrance, providing depth and complexity. Some popular middle notes include rose, geranium, or caraway. These notes will help to further develop the scent profile and provide a sense of projection.
3. Base notes (the 30-hour mark and beyond): These are the richest, most lingering notes in the fragrance. Consider including deeper, more sensual ingredients like sandalwood, vanilla, or amber to create a warm and lasting finish.

Now, based on your preference

In [27]:
text = "Picea Abies,Cedrus Atlantica, Cananga Odorata, Citrus, vetiver. design my fragrance using these ingredients."
text = "Make a detaled blend for a 100Ml batch "
response = llm_chain.run(text)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  As your dedicated perfume design assistant, I'm thrilled to help you create a detailed blend for a 100ML batch. To ensure we craft the perfect fragrance, please provide me with the following information:

1. Essence or mood you want to convey: Describe the emotions or atmosphere you want your perfume to evoke. For example, do you want it to be fresh, floral, spicy, or sweet?
2. Ingredients you want to include: List the specific fragrance notes you're interested in using. These can be natural essential oils, aroma compounds, or a combination of both. Examples might include lavender, vanilla, rose, or peppermint.
3. Level of intensity desired: Do you want your perfume to be light and airy or rich and enveloping? This will help us determine the appropriate balance of fragrance notes.
4. Length of the blend: Would you like the perfume to be long-lasting or have a shorter duration?

Once I have this information, I can provide personalized suggestions and recommendations to help you create

In [28]:
text = "formulate the fragrance "
response = llm_chain.run(text)
print(response)

  Of course! As your dedicated perfume design assistant, I'm delighted to help you formulate a fragrance that captures the essence or mood you're aiming for. Here's a step-by-step guide to get you started:

Step 1: Identify the Mood or Essence You Want to Convey

Before we begin, can you tell me what mood or essence you want to convey through your fragrance? For example, do you want it to be bright and uplifting, warm and comforting, or sophisticated and seductive? Narrowing down the mood or essence will help me provide more tailored suggestions.

Step 2: Explore Ingredients That Will Help You Achieve Your Desired Mood or Essence

Once you have a clear mood or essence in mind, we can start exploring fragrance ingredients that will help you achieve it. There are hundreds of fragrance ingredients to choose from, each with its unique characteristics and properties. I can help you identify the best ingredients to create a fragrance that meets your desired mood or essence.

Step 3: Consider